# Computing Customer Lifetime Value

CLTV = ((Average Order Value * Purchase Frequency)/Churn Rate) * Profit Margin

### Preprate the Data with Customer Grouping

In [1]:
import pandas as pd
import numpy as np 
import warnings 
warnings.filterwarnings('ignore')
df = pd.read_csv('Ecommerce_Data.csv')
df.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Hour
0,439570,574477,22591,CARDHOLDER GINGHAM CHRISTMAS TREE,1,3.25,15453.0,United Kingdom,2011-11-04,12
1,387281,570275,23541,WALL ART CLASSIC PUDDINGS,12,7.45,13098.0,United Kingdom,2011-10-10,10
2,337863,566482,22508,DOORSTOP RETROSPOT HEART,12,3.75,16609.0,United Kingdom,2011-09-13,9
3,57628,541215,22662,LUNCH BAG DOLLY GIRL DESIGN,10,1.65,14329.0,United Kingdom,2011-01-14,13
4,330897,565930,POST,POSTAGE,5,18.00,12685.0,France,2011-09-08,10


In [2]:
# some small data wrangling task
df['Total_Revenue']=df['UnitPrice']*df['Quantity']
df.drop('Unnamed: 0', axis=1,inplace = True)
df['Date']=pd.to_datetime(df['Date'])
df = df[df['Quantity']>0]
df.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Hour,Total_Revenue
0,574477,22591,CARDHOLDER GINGHAM CHRISTMAS TREE,1,3.25,15453.0,United Kingdom,2011-11-04,12,3.25
1,570275,23541,WALL ART CLASSIC PUDDINGS,12,7.45,13098.0,United Kingdom,2011-10-10,10,89.40
2,566482,22508,DOORSTOP RETROSPOT HEART,12,3.75,16609.0,United Kingdom,2011-09-13,9,45.00
3,541215,22662,LUNCH BAG DOLLY GIRL DESIGN,10,1.65,14329.0,United Kingdom,2011-01-14,13,16.50
4,565930,POST,POSTAGE,5,18.00,12685.0,France,2011-09-08,10,90.00


In [3]:
#lets create grouping around customer ID and aggregate the metrics 
max_date = df['Date'].max()
max_date

Timestamp('2011-12-09 00:00:00')

In [4]:
df2 = df.groupby(['CustomerID']).agg(
    {
        'Date': lambda x: (max_date - x.min()).days,
        'InvoiceNo': lambda x: len(x),
        'Quantity': lambda x: x.sum(),
        'Total_Revenue': lambda x: x.sum()
     
     }
)
df2

,Date,InvoiceNo,Quantity,Total_Revenue
CustomerID,,,,
12347.0,367,48,623,1146.59
12348.0,358,7,531,550.92
12349.0,18,16,203,406.32
12350.0,310,2,36,45.60
12352.0,296,21,163,663.33
...,...,...,...,...
18280.0,277,3,9,52.75
18281.0,180,2,11,33.45
18282.0,126,4,11,67.85


In [5]:
#lets put this all together in a data function 

def customer_model(data):
    max_date = data['Date'].max()
    data = data.groupby('CustomerID').agg({
            'Date': lambda x: (max_date - x.min()).days,
            'InvoiceNo': lambda x: len(x),
            'Quantity': lambda x: x.sum(),
            'Total_Revenue': lambda x: x.sum()
    })
    return data     
    
data = customer_model(df)
data.head()

,Date,InvoiceNo,Quantity,Total_Revenue
CustomerID,,,,
12347.0,367,48,623,1146.59
12348.0,358,7,531,550.92
12349.0,18,16,203,406.32
12350.0,310,2,36,45.60
12352.0,296,21,163,663.33


In [6]:
#change the name of th columns and insure that we don't have 0 quantity 

data.columns = ['age', 'num_transactions', 'quantity', 'total_revenue']
data.head()

,age,num_transactions,quantity,total_revenue
CustomerID,,,,
12347.0,367,48,623,1146.59
12348.0,358,7,531,550.92
12349.0,18,16,203,406.32
12350.0,310,2,36,45.60
12352.0,296,21,163,663.33


In [7]:
data = data[data['quantity']>0]
data.head()


,age,num_transactions,quantity,total_revenue
CustomerID,,,,
12347.0,367,48,623,1146.59
12348.0,358,7,531,550.92
12349.0,18,16,203,406.32
12350.0,310,2,36,45.60
12352.0,296,21,163,663.33


In [8]:
data[data['quantity']==0]

,age,num_transactions,quantity,total_revenue
CustomerID,,,,


## Getting the Elements of CLTV Calculations

In [9]:
data['AOV'] = data['total_revenue']/data['num_transactions']
data.head()

,age,num_transactions,quantity,total_revenue,AOV
CustomerID,,,,,
12347.0,367,48,623,1146.59,23.887292
12348.0,358,7,531,550.92,78.702857
12349.0,18,16,203,406.32,25.395000
12350.0,310,2,36,45.60,22.800000
12352.0,296,21,163,663.33,31.587143


In [10]:
#lets calculate purchase frequency
purchase_freq = sum(data['num_transactions'])/len(data)
purchase_freq


19.496336101612115

In [11]:
data.shape[0]

4094

In [12]:
repeat_rate = data[data['num_transactions']>1].shape[0]/data.shape[0]
repeat_rate

0.9291646311675623

In [13]:
churn_rate = 1 -repeat_rate
churn_rate

0.07083536883243768

In [14]:
#choose whatever specifications you want for profit margin, we went with 10 %
data['profit_margin'] = data['total_revenue'] * 0.10
data.head()

,age,num_transactions,quantity,total_revenue,AOV,profit_margin
CustomerID,,,,,,
12347.0,367,48,623,1146.59,23.887292,114.659
12348.0,358,7,531,550.92,78.702857,55.092
12349.0,18,16,203,406.32,25.395000,40.632
12350.0,310,2,36,45.60,22.800000,4.560
12352.0,296,21,163,663.33,31.587143,66.333


## Putting the Equation Together

In [15]:
# import seaborn
import seaborn as sns

In [18]:
data['CLTV'] = ((data['AOV']*purchase_freq)/churn_rate)*.10
data.head()

,age,num_transactions,quantity,total_revenue,AOV,profit_margin,CLTV
CustomerID,,,,,,,
12347.0,367,48,623,1146.59,23.887292,114.659,657.460637
12348.0,358,7,531,550.92,78.702857,55.092,2166.174018
12349.0,18,16,203,406.32,25.395000,40.632,698.957969
12350.0,310,2,36,45.60,22.800000,4.560,627.534621
12352.0,296,21,163,663.33,31.587143,66.333,869.387093


In [19]:
print("The median CLTV is ", data['CLTV'].median())
print("The average CLTV is ", data['CLTV'].mean())


The median CLTV is  460.5410131773401
The average CLTV is  747.7434697480141


In [20]:
data.sort_values(by='CLTV').reset_index()

,CustomerID,age,num_transactions,quantity,total_revenue,AOV,profit_margin,CLTV
0,12977.0,156,1,2,0.78,0.780000,0.078,21.468290
1,14661.0,26,1,1,0.85,0.850000,0.085,23.394931
2,16658.0,368,1,1,0.85,0.850000,0.085,23.394931
3,17962.0,142,3,3,2.99,0.996667,0.299,27.431703
4,17859.0,304,5,5,5.21,1.042000,0.521,28.679433
...,...,...,...,...,...,...,...,...
4089,12931.0,239,15,5590,9344.82,622.988000,934.482,17146.777994
4090,16333.0,298,11,7840,7944.40,722.218182,794.440,19877.934771
4091,17949.0,289,10,4565,7671.45,767.145000,767.145,21114.475728
4092,16000.0,2,5,2630,7948.30,1589.660000,794.830,43752.924786
